In [39]:
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime as datetime
import os
import sys
import numpy as np
import plotly.express as px


In [40]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'FPL')))


In [41]:
from FPL.fpl_api_collection import (
    get_bootstrap_data,
get_current_gw,
get_fixt_dfs,
get_league_table,
get_fixture_dfs,
    get_fixture_data
)
from FPL.fpl_utils import (
    define_sidebar,
    get_annot_size,
    map_float_to_color,
    get_text_color_from_hash,
    get_rotation,
    get_user_timezone
)
from FPL.fpl_params import (
    TIMEZONES_BY_CONTINENT,
    AUTHOR_CONTINENT,
    AUTHOR_CITY
)

In [42]:
team_fdr_df, team_fixt_df, team_ga_df, team_gf_df = get_fixt_dfs()

events_df = pd.DataFrame(get_bootstrap_data()['events'])

gw_min = min(events_df['id'])
gw_max = max(events_df['id'])

ct_gw = get_current_gw()

In [43]:
fixt = team_fixt_df.reset_index()
drf = team_fdr_df.reset_index()
ga = team_ga_df.reset_index()
gf = team_gf_df.reset_index()

# Rename the first column to 'Team'
fixt.rename(columns={0: 'Team'}, inplace=True)
drf.rename(columns={0: 'Team'}, inplace=True)
ga.rename(columns={0: 'Team'}, inplace=True)
gf.rename(columns={0: 'Team'}, inplace=True)

In [44]:
merged_df = pd.merge(fixt, drf, on="Team", suffixes=('_Fixture', '_FDR'))

# Convert to a format suitable for display
merged_display_df = merged_df.copy()
merged_display_df.columns = ["Team"] + [f"{gw}_Fixture" for gw in merged_display_df.columns[1:4]] + [f"{gw}_FDR" for gw in merged_display_df.columns[4:]]

# Streamlit app
st.title("FPL Fixture and Difficulty Rating Analysis")

# Display the merged DataFrame
st.markdown("### Fixture Details with Fixture Difficulty Ratings")
st.dataframe(merged_display_df)

2024-10-17 20:21:53.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:21:53.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:21:53.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:21:53.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:21:53.595 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:21:53.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [45]:
saaaa=get_fixture_data()

In [46]:
import json

fixtures_df = pd.DataFrame(saaaa)

fixtures_df.drop(columns='stats', inplace=True)


fixtures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   code                    380 non-null    int64  
 1   event                   380 non-null    int64  
 2   finished                380 non-null    bool   
 3   finished_provisional    380 non-null    bool   
 4   id                      380 non-null    int64  
 5   kickoff_time            380 non-null    object 
 6   minutes                 380 non-null    int64  
 7   provisional_start_time  380 non-null    bool   
 8   started                 380 non-null    bool   
 9   team_a                  380 non-null    int64  
 10  team_a_score            70 non-null     float64
 11  team_h                  380 non-null    int64  
 12  team_h_score            70 non-null     float64
 13  team_h_difficulty       380 non-null    int64  
 14  team_a_difficulty       380 non-null    in

In [47]:
teams_df = pd.DataFrame(get_bootstrap_data()['teams'])

In [49]:
time=get_user_timezone()
saaaa=get_fixture_data()
fixtures_df = pd.DataFrame(saaaa)
fixtures_df.drop(columns='stats', inplace=True)
teams_df = pd.DataFrame(get_bootstrap_data()['teams'])
teams_df['logo_url'] = "https://resources.premierleague.com/premierleague/badges/70/t" + teams_df['code'].astype(str) + ".png"
team_name_mapping = pd.Series(teams_df.name.values, index=teams_df.id).to_dict()
fixtures_df = fixtures_df.merge(teams_df[['id', 'logo_url']], left_on='team_h', right_on='id', how='left').rename(columns={'logo_url': 'team_h_logo'})
fixtures_df = fixtures_df.merge(teams_df[['id', 'logo_url']], left_on='team_a', right_on='id', how='left').rename(columns={'logo_url': 'team_a_logo'})
fixtures_df['team_a'] = fixtures_df['team_a'].replace(team_name_mapping)
fixtures_df['team_h'] = fixtures_df['team_h'].replace(team_name_mapping)
fixtures_df = fixtures_df.drop(columns=['pulse_id'])
fixtures_df['datetime'] = pd.to_datetime(fixtures_df['kickoff_time'], utc=True)
fixtures_df['local_time'] = fixtures_df['datetime'].dt.tz_convert(time).dt.strftime('%A %d %B %Y %H:%M')
fixtures_df['local_date'] = fixtures_df['datetime'].dt.tz_convert(time).dt.strftime('%d %A %B %Y')
fixtures_df['local_hour'] = fixtures_df['datetime'].dt.tz_convert(time).dt.strftime('%H:%M')


In [51]:
teams_df

,code,draw,form,id,loss,name,played,points,position,short_name,...,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id,logo_url
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,False,0,1350,1375,1360,1360,1340,1390,1,https://resources.premierleague.com/premierlea...
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,False,0,1160,1220,1120,1140,1200,1300,2,https://resources.premierleague.com/premierlea...
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,False,0,1100,1145,1070,1130,1130,1160,127,https://resources.premierleague.com/premierlea...
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,False,0,1090,1155,1080,1120,1100,1190,130,https://resources.premierleague.com/premierlea...
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,False,0,1100,1175,1070,1130,1130,1220,131,https://resources.premierleague.com/premierlea...
5,8,0,None,6,0,Chelsea,0,0,0,CHE,...,False,0,1225,1230,1130,1130,1320,1330,4,https://resources.premierleague.com/premierlea...
6,31,0,None,7,0,Crystal Palace,0,0,0,CRY,...,False,0,1095,1095,1100,1100,1090,1090,6,https://resources.premierleague.com/premierlea...
7,11,0,None,8,0,Everton,0,0,0,EVE,...,False,0,1080,1100,1080,1100,1080,1100,7,https://resources.premierleague.com/premierlea...
8,54,0,None,9,0,Fulham,0,0,0,FUL,...,False,0,1100,1160,1100,1140,1100,1180,34,https://resources.premierleague.com/premierlea...
9,40,0,None,10,0,Ipswich,0,0,0,IPS,...,False,0,1060,1090,1070,1080,1050,1100,8,https://resources.premierleague.com/premierlea...


In [3]:
from mplsoccer.pitch import Pitch
pitch = Pitch(pitch_color="green",corner_acs=True,half=True,line_color='white', stripe=True)
fig, ax = pitch.draw()

TypeError: BasePitch.__init__() got an unexpected keyword argument 'corner_acs'